In [8]:
import pandas as pd
import streamlit as st
import folium
from folium.plugins import HeatMap
import plotly.express as px
from plotly.offline import plot

from script.modules.gcp_class import Bigquery_cliant

bq = Bigquery_cliant()
table_id = st.secrets["bigquery"]["table_id"]

In [17]:
df = bq.read_sql(f"select * from  {table_id}")

In [4]:
# ヒートマップ
vis = df.groupby(["longitude", "latitude"])["tokyo_crimes_id"].count().rename("犯罪件数").reset_index()
folium_map = folium.Map(location=[35.74, 139.70], zoom_start=10)
HeatMap(
    data=vis[["longitude", "latitude", "犯罪件数"]].values.tolist(),
    radius=15,
).add_to(folium_map)
folium_map

In [12]:
# 時系列推移
vis = df.groupby(["occurrence_month", "teguchi"])["tokyo_crimes_id"].count().rename("犯罪件数").reset_index()
# long_df = px.data.medals_long()
fig = px.bar(
    vis, 
    x="occurrence_month",
    y="犯罪件数",
    color="teguchi",
    title="窃盗犯罪件数",
    color_discrete_sequence=px.colors.qualitative.Set1)
# X軸の日付形式を変更
fig.update_xaxes(tickformat="%Y年%m月")  # 例: 2022-11 の形式

plot(fig)

'temp-plot.html'

In [14]:
# 区ごとの件数
vis = df.groupby("city")["tokyo_crimes_id"].count().rename("犯罪件数").reset_index().sort_values("犯罪件数")
# long_df = px.data.medals_long()
fig = px.bar(
    vis, 
    y="city",
    x="犯罪件数",
    orientation='h',  # 横向きの棒グラフに設定
    title="窃盗犯罪件数",
    color_discrete_sequence=['blue']  # 単色（青）に設定
    )
# Y軸の目盛りを調整してすべて表示
fig.update_yaxes(
    title="",
    automargin=True,
    dtick=1  # 1つごとのデータポイントで目盛りを表示
)
plot(fig)

'temp-plot.html'

In [15]:


vis = df.groupby(["day_of_week", "occurrence_time_str"])["tokyo_crimes_id"].count().rename("犯罪件数").reset_index()
# ヒートマップの作成
fig = px.density_heatmap(
    vis, 
    x="occurrence_time_str", 
    y="day_of_week", 
    z="犯罪件数",  # カウント数で色の濃淡を表現
    color_continuous_scale='Viridis',  # 色のスケールを指定
    title="曜日別・時間帯別窃盗犯罪件数",
    category_orders={
        "day_of_week": ["月", "火", "水", "木", "金", "土", "日"],
    }
)

plot(fig)

/var/folders/t7/j7l9x5bn1n361lz4t8rrxj_h0000gn/T/ipykernel_86310/4122834833.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



'temp-plot.html'

In [17]:
tf = ~df["victim_sex"].isin(["不明", "法人・団体"])
vis = df[tf].groupby(["victim_age", "victim_sex"])["tokyo_crimes_id"].count().rename("犯罪件数").reset_index()
fig = px.bar(
    vis, 
    x="victim_sex",
    y="犯罪件数",
    color="victim_age",
    title="窃盗犯罪件数",
    color_discrete_sequence=px.colors.sequential.Blues,
    category_orders={"victim_age": ["10歳未満", "10歳代", "20歳代", "30歳代", "40歳代", "50歳代", "60-64歳", "65-69歳", "70歳以上", "法人・団体、被害者なし"]}
    
)
plot(fig)

'temp-plot.html'

In [25]:
df_round

,precipitation_level,average_temperature_level,occurrence_date,犯罪件数
0,0mm,10-19度,2018-04-01,164
1,0mm,10-19度,2018-04-02,135
2,0mm,10-19度,2018-04-03,115
3,0mm,10-19度,2018-04-05,125
4,0mm,10-19度,2018-04-07,134
...,...,...,...,...
1638,50mm以上,20-24度,2022-09-20,70
1639,50mm以上,20-24度,2022-09-24,61
1640,50mm以上,25度以上,2019-09-09,116
1641,50mm以上,25度以上,2021-08-08,45


In [5]:
# データフレームのサンプルを作成
import pandas as pd
df = pd.DataFrame({"A":[1, 2, 3]})


KeyError: True

In [24]:
df_round = df.groupby(["precipitation_level", "average_temperature_level", "occurrence_date"])["tokyo_crimes_id"].count().rename("犯罪件数").reset_index()
vis = df_round.groupby(["precipitation_level", "average_temperature_level"])["犯罪件数"].mean().reset_index()

# ヒートマップの作成
fig = px.density_heatmap(
    vis, 
    x="precipitation_level", 
    y="average_temperature_level", 
    z="犯罪件数",  # カウント数で色の濃淡を表現
    color_continuous_scale='Viridis',  # 色のスケールを指定
    # title="曜日別・時間帯別窃盗犯罪件数",
    category_orders={
        "average_temperature_level": ["10度以下", "10-19度", "20-24度", "25度以上"],
    }
)

plot(fig)

'temp-plot.html'

In [19]:
df_round = df.groupby(["precipitation_level", "occurrence_date"])["tokyo_crimes_id"].count().rename("犯罪件数").reset_index()
vis = df_round.groupby("precipitation_level")["犯罪件数"].mean().reset_index()
fig = px.bar(
    vis, 
    x="precipitation_level",
    y="犯罪件数",
    title="降水量と窃盗犯罪平均件数",
    color_discrete_sequence=['blue']  # 単色（青）に設定
)
# y軸のタイトルを変更
fig.update_yaxes(title="1日あたりの件数")
fig.update_xaxes(title="1日の降水量")
plot(fig)

'temp-plot.html'